# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks =  pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 90871227, 'calculationPrice': 'tops', 'change': -3.71, 'changePercent': -0.0227, 'close': 0, 'closeSource': 'fcofiail', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 163.17, 'iexAskSize': 408, 'iexBidPrice': 164.56, 'iexBidSize': 313, 'iexClose': 158.52, 'iexCloseTime': 1664643789847, 'iexLastUpdated': 1714658155925, 'iexMarketPercent': 0.012322194140523202, 'iexOpen': 159.73, 'iexOpenTime': 1681581839552, 'iexRealtimePrice': 161.76, 'iexRealtimeSize': 2, 'iexVolume': 497021, 'lastTradeTime': 1650001756258, 'latestPrice': 163.56, 'latestSource': 'IEX real time price', 'latestTime': '11:55:27 AM', 'latestUpdate': 1698952855275, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2652060547062, 'oddLotDelaye

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

14.58

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [4]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [5]:
final_dataframe = pd.DataFrame(columns  = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data.status_code)
    for symbol in symbol_string.split(','):
        if symbol == 'WLTW': continue
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A',
            
            
        ], 
            index = my_columns
        ), 
            ignore_index = True
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,136.590,17.05,N/A
1,AAL,16.960,-3.32,N/A
2,AAP,232.820,24.12,N/A
3,AAPL,164.523,14.28,N/A
4,ABBV,133.640,33.1,N/A
...,...,...,...,...
499,YUM,128.510,24.51,N/A
500,ZBH,125.320,32.58,N/A
501,ZBRA,484.010,31.19,N/A
502,ZION,67.600,6.35,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [6]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace= True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0 ]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.head()

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,63,BIO,575.082,2.63,N/A
1,225,HPE,15.991,3.08,N/A
2,152,EBAY,59.390,3.17,N/A
3,108,COO,398.450,3.31,N/A
4,226,HPQ,35.089,3.39,N/A


In [7]:
final_dataframe.drop('index', axis= 1 ,inplace = True)
final_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,575.082,2.63,N/A
1,HPE,15.991,3.08,N/A
2,EBAY,59.390,3.17,N/A
3,COO,398.450,3.31,N/A
4,HPQ,35.089,3.39,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,575.082,2.63,347
1,HPE,15.991,3.08,12507
2,EBAY,59.390,3.17,3367
3,COO,398.450,3.31,501
4,HPQ,35.089,3.39,5699


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats']['enterpriseValue'])
#data.status_code


# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']


# Price-to-book ratio

pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio

ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda_ratio = enterprise_value/ebitda
print(ev_to_ebitda_ratio)

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
print(gross_profit)

ev_to_gross_profit_ratio = enterprise_value / gross_profit
print(ev_to_gross_profit_ratio)

11.187919562469954
311116366322
8.917489131303265


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earning Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EDITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
    
]


rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    #print(data)
    for symbol in symbol_string.split(','):
        if symbol == 'WLTW': continue
        #pull value first, not all stocks have them defined
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN


            
        rv_dataframe = rv_dataframe.append(
        pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N?A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            
        ], index = rv_columns),
        ignore_index = True)
        

In [13]:
rv_dataframe.head()
    
    

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EDITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.270,N?A,17.44,N/A,4.02,N/A,3.4,N/A,12.753765,N/A,6.286433,N/A,N/A
1,AAL,16.768,N?A,-3.38,N/A,-1.47,N/A,0.435,N/A,-7.322785,N/A,1.404391,N/A,N/A
2,AAP,238.580,N?A,24.32,N/A,4.55,N/A,1.33,N/A,13.759770,N/A,3.109722,N/A,N/A
3,AAPL,162.670,N?A,14.75,N/A,20.97,N/A,3.76,N/A,11.204526,N/A,9.145671,N/A,N/A
4,ABBV,138.340,N?A,32.12,N/A,17.4,N/A,4.35,N/A,11.260972,N/A,7.725696,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EDITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,185.200,N?A,60.47,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,280.770,N?A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,311.040,N?A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N?A,9.75,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.207,N?A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.650,N?A,14.8,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.130,N?A,32.06,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,36.893,N?A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,125.710,N?A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2657.660,N?A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [16]:
# for column in rv_dataframe.columns:
#     rv_dataframe[column].fillna(rv_dataframe[column].mean())

In [17]:
for column in [
    'Price-to-Earning Ratio',
    'Price-to-Book Ratio',
    'Price-to-Sales Ratio',
    'EV/EDITDA',
    'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EDITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [19]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earning Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EDITDA': 'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric] , rv_dataframe.loc[row, metric]) / 100

In [20]:
rv_dataframe.head(20)

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EDITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.270,N?A,17.44,0.35119,4.0200,0.553571,3.400,0.530754,12.753765,0.424603,6.286433,0.46627,N/A
1,AAL,16.768,N?A,-3.38,0.0555556,-1.4700,0.0555556,0.435,0.0357143,-7.322785,0.0238095,1.404391,0.0575397,N/A
2,AAP,238.580,N?A,24.32,0.537698,4.5500,0.60119,1.330,0.190476,13.759770,0.478175,3.109722,0.176587,N/A
3,AAPL,162.670,N?A,14.75,0.291667,20.9700,0.938492,3.760,0.581349,11.204526,0.363095,9.145671,0.676587,N/A
4,ABBV,138.340,N?A,32.12,0.680556,17.4000,0.922619,4.350,0.636905,11.260972,0.365079,7.725696,0.561508,N/A
5,ABC,130.000,N?A,8.83,0.150794,62.4300,0.984127,0.066,0.00396825,5.191161,0.0793651,2.476360,0.126984,N/A
6,ABMD,309.320,N?A,93.62,0.960317,10.6700,0.837302,14.940,0.948413,46.380595,0.962302,16.201393,0.886905,N/A
7,ABT,125.630,N?A,31.33,0.660714,6.3900,0.728175,5.370,0.740079,21.703246,0.730159,9.453971,0.690476,N/A
8,ACN,342.200,N?A,35.50,0.749008,11.4900,0.853175,4.240,0.626984,21.662810,0.724206,12.288870,0.801587,N/A
9,ADBE,526.090,N?A,42.43,0.813492,17.3700,0.920635,16.700,0.96627,43.721531,0.956349,18.872717,0.934524,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [21]:
from statistics import mean

for row in rv_dataframe.index:
#     pe_ratio =
#     pb_ratio =
#     ps_ratio = 
#     ev_to_ebitda = 
#     ev_to_gp = 
    value_percentile  =   []
    for metric in metrics.keys():
        value_percentile.append(rv_dataframe.loc[row, metrics[metric]])
    
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentile)
#rv_dataframe

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [22]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EDITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.768,N?A,-3.38,0.0555556,-1.4700,0.0555556,0.4350,0.0357143,-7.322785,0.0238095,1.404391,0.0575397,0.0456349
1,HPQ,34.980,N?A,3.31,0.0714286,-11.7900,0.0376984,0.3115,0.0119048,3.797763,0.0416667,1.641152,0.0793651,0.0484127
2,UNM,26.220,N?A,6.70,0.111111,0.4783,0.0575397,0.3959,0.031746,3.148211,0.0357143,0.396030,0.00793651,0.0488095
3,PRU,111.754,N?A,6.10,0.0972222,0.6738,0.0674603,0.5966,0.0575397,4.490940,0.0634921,0.577304,0.0119048,0.0595238
4,BEN,32.550,N?A,4.46,0.0793651,0.7365,0.0753968,0.9890,0.119048,2.920003,0.0337302,0.806019,0.0218254,0.065873
5,DHI,92.500,N?A,3.88,0.0753968,1.1300,0.112103,0.5924,0.0555556,3.181424,0.0376984,2.179710,0.117063,0.0795635
6,BA,204.140,N?A,-14.27,0.0436508,-8.3600,0.0456349,2.0000,0.328373,-34.396389,0.00595238,-1391.182872,0.00198413,0.085119
7,HPE,16.167,N?A,3.17,0.0674603,0.5259,0.0615079,0.3814,0.0257937,7.774336,0.18254,2.076184,0.107143,0.0888889
8,AIG,59.620,N?A,9.16,0.154762,0.7564,0.0793651,1.0000,0.123016,4.387334,0.0615079,1.009991,0.031746,0.0900794
9,AIZ,150.780,N?A,6.85,0.117063,1.5000,0.1875,0.8650,0.105159,1.882556,0.0277778,0.843613,0.0238095,0.0922619


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])

rv_dataframe.head()
    

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('value strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
writer.save()

## Saving Our Excel Output
As before, saving our Excel output is very easy: